In [1]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

2.5.1
CUDA available: True
GPU name: NVIDIA GeForce RTX 4090


In [8]:
!pip install torch torchvision
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ================================
# Paths (update if needed)
# ================================
base_dir = r"C:\Users\Ruijia Li\Downloads\archive"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
meta_dir = os.path.join(base_dir, "meta")

# ================================
# Transforms
# ================================
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ================================
# Dataset & Dataloader
# ================================
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ================================
# CNN Model
# ================================
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# ================================
# Model Setup
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ================================
# Training Loop
# ================================
print("Start training")
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

# ================================
# Save the model
# ================================
torch.save(model.state_dict(), "simple_cnn_model.pth")
print("Training complete and model saved.")


Start training
Epoch 1/50, Loss: 0.6578
Epoch 2/50, Loss: 0.0854
Epoch 3/50, Loss: 0.0432
Epoch 4/50, Loss: 0.0398
Epoch 5/50, Loss: 0.0282
Epoch 6/50, Loss: 0.0287
Epoch 7/50, Loss: 0.0263
Epoch 8/50, Loss: 0.0216
Epoch 9/50, Loss: 0.0215
Epoch 10/50, Loss: 0.0272
Epoch 11/50, Loss: 0.0194
Epoch 12/50, Loss: 0.0125
Epoch 13/50, Loss: 0.0175
Epoch 14/50, Loss: 0.0148
Epoch 15/50, Loss: 0.0161
Epoch 16/50, Loss: 0.0207
Epoch 17/50, Loss: 0.0163
Epoch 18/50, Loss: 0.0150
Epoch 19/50, Loss: 0.0101
Epoch 20/50, Loss: 0.0155
Epoch 21/50, Loss: 0.0166
Epoch 22/50, Loss: 0.0128
Epoch 23/50, Loss: 0.0161
Epoch 24/50, Loss: 0.0134
Epoch 25/50, Loss: 0.0161
Epoch 26/50, Loss: 0.0121
Epoch 27/50, Loss: 0.0152
Epoch 28/50, Loss: 0.0172
Epoch 29/50, Loss: 0.0113
Epoch 30/50, Loss: 0.0140
Epoch 31/50, Loss: 0.0235
Epoch 32/50, Loss: 0.0156
Epoch 33/50, Loss: 0.0145
Epoch 34/50, Loss: 0.0200
Epoch 35/50, Loss: 0.0054
Epoch 36/50, Loss: 0.0151
Epoch 37/50, Loss: 0.0178
Epoch 38/50, Loss: 0.0256
Epoch 

In [10]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from PIL import Image

# =========================
# Paths
# =========================
train_dir = r"C:\Users\Ruijia Li\Downloads\archive\train"
test_csv = r"C:\Users\Ruijia Li\Downloads\archive\Test.csv"
model_path = "simple_cnn_model.pth"

# =========================
# Transform
# =========================
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# =========================
# Load training data to get class mapping
# =========================
train_dataset = datasets.ImageFolder(train_dir)
class_to_idx = train_dataset.class_to_idx      # e.g. {'0': 0, '1': 1, ..., '42': 42}
idx_to_class = {v: k for k, v in class_to_idx.items()}  # reverse mapping

num_classes = len(class_to_idx)

# =========================
# Your CNN model
# =========================
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# =========================
# Load model
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# =========================
# Load Test.csv
# =========================
test_df = pd.read_csv(test_csv)

# =========================
# Predict + Accuracy
# =========================
correct = 0
total = 0

with torch.no_grad():
    for _, row in test_df.iterrows():
        path = row["Path"]                   # e.g., Test/00001.png
        raw_label = str(row["ClassId"])      # e.g., '16'
        true_label = class_to_idx.get(raw_label, None)  # Convert to model index

        if true_label is None:
            continue  # skip unknown class

        # Full image path
        image_path = os.path.join(r"C:\Users\Ruijia Li\Downloads\archive", path)

        if not os.path.exists(image_path):
            continue

        image = Image.open(image_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        output = model(input_tensor)
        pred_label = torch.argmax(output, dim=1).item()

        total += 1
        if pred_label == true_label:
            correct += 1

accuracy = 100 * correct / total
print(f"\n✅ Final Test Accuracy: {accuracy:.2f}% ({correct}/{total})")


C:\Users\Ruijia Li\AppData\Local\Temp\ipykernel_19416\2628976350.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati


✅ Final Test Accuracy: 94.71% (11962/12630)


In [11]:
torch.save(model.state_dict(), r"C:\Users\Ruijia Li\Downloads\archive\simple_cnn_model.pth")
